In [5]:
import pandas as pd
import numpy as np
import mysql.connector
from datetime import timedelta
from datetime import date

In [43]:
class Form:
    def __init__(self, ftg, ftr, htg, htr, sht_cor):
        self.ftg = ftg    # full time goals - tuple: (average goals for, average goals against)
        self.ftr = ftr    # full time results - tuple: (win proportin, draw proportion, loss proportion)
        self.htg = htg    # half time goals - tuple
        self.htr = htr    # half time results - tuple
        self.sht_cor = sht_cor # shots and corners - tuple: (shots on target for, sot against, shots for, s against, corners for, c against)
        

In [44]:
class Team:
    def __init__(self, name, home_form, away_form):
        self.name = name
        self.home_form = home_form
        self.away_form = away_form

In [48]:
def team_averages_per_game_and_runs(team, opp_team, this_match_date, earliest_date, at_home, df, min_games):
    if at_home == True:
        adj = 0 # adjuster
        h_or_a = 'HomeTeam'
    else:
        adj = 1
        h_or_a = 'AwayTeam'
        
    
    df_team = df.loc[(df['Date']>earliest_date) & (df['Date']<this_match_date) & (df[h_or_a]==team)]
    
    if len(df_team) >= min_games:
        team_averages_and_runs = {
            'ftg_for': 0.0,  # full time goals for
            'ftg_ag': 0.0,   # full time goals against
            'ftw': 0.0,      # full time win
            'ftd': 0.0,      # full time draw
            'ftl': 0.0,      # full time loss
            'htg_for': 0.0,  # half time goals for
            'htg_ag': 0.0,   # half time goals against
            'htw': 0.0,      # half time win
            'htd': 0.0,      # half time draw
            'htl': 0.0,      # half time loss
            'sot_for': 0.0,  # shots on target for
            'sot_ag': 0.0,   # shots on target against
            's_for': 0.0,    # shots for
            's_ag': 0.0,     # shots against
            'c_for': 0.0,    # corners for
            'c_ag': 0.0,     # corners against
            'win_run': 0,    # runs
            'winless_run': 0, 
            'draw_run': 0,
            'drawless_run': 0,
            'losing_run': 0,
            'lossless_run': 0
        }
        pld_weight = 0.0 # number of games played represented as time-weighted units


        for i in range (0, len(df_team)):
            that_match_date = df_team.iloc[i]['Date'] # date of previous match
            days_gap = float((this_match_date - that_match_date).days)
            time_coeff = np.exp(-0.007 * days_gap)
            relevance = time_coeff
            if df_team.iloc[i, 3-adj] == opp_team: #is this previous match's opposition the same as today's?
                relevance *= 1.5
            pld_weight += relevance

            team_averages_and_runs['ftg_for'] += (df_team.iloc[i, 4+adj] * relevance)
            team_averages_and_runs['ftg_ag'] += (df_team.iloc[i, 5-adj] * relevance)
            if (df_team.iloc[i, 6]=='H' and at_home==True) or (df_team.iloc[i, 6]=='A' and at_home==False):
                team_averages_and_runs['ftw'] += relevance                    
            elif df_team.iloc[i, 6] == 'D':
                team_averages_and_runs['ftd'] += relevance                    
            elif (df_team.iloc[i, 6]=='A' and at_home==True) or (df_team.iloc[i, 6]=='H' and at_home==False): # perhaps just 'else' but this feels right
                team_averages_and_runs['ftl'] += relevance                    
            team_averages_and_runs['htg_for'] += (df_team.iloc[i, 7+adj] * relevance)  
            team_averages_and_runs['htg_ag'] += (df_team.iloc[i, 8-adj] * relevance)  
            if (df_team.iloc[i, 9]=='H' and at_home==True) or (df_team.iloc[i, 9]=='A' and at_home==False):
                team_averages_and_runs['htw'] += relevance                    
            elif df_team.iloc[i, 9] == 'D':
                team_averages_and_runs['htd'] += relevance                    
            elif (df_team.iloc[i, 9]=='A' and at_home==True) or (df_team.iloc[i, 9]=='H' and at_home==False):
                team_averages_and_runs['htl'] += relevance                    
            team_averages_and_runs['sot_for'] += (df_team.iloc[i, 10+adj] * relevance)
            team_averages_and_runs['sot_ag'] += (df_team.iloc[i, 11-adj] * relevance)
            team_averages_and_runs['s_for'] += (df_team.iloc[i, 12+adj] * relevance)
            team_averages_and_runs['s_ag'] += (df_team.iloc[i, 13-adj] * relevance)
            team_averages_and_runs['c_for'] += (df_team.iloc[i, 14+adj] * relevance)
            team_averages_and_runs['c_ag'] += (df_team.iloc[i, 15-adj] * relevance)

        # divide
        tm_av_key_list = list(team_averages_and_runs.keys())
        for j in range(0, len(tm_av_key_list)):
            team_averages_and_runs[tm_av_key_list[j]] /= pld_weight
        return team_averages_and_runs
    else:
        return None

In [27]:
conn_str = mysql.connector.connect(
    host = 'localhost',
    user = 'simon',
    password = 'chainsaw',
    database = 'football'
)


query = "SELECT ThisDiv, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR, HST, AwST, HS, AwS, HC, AC FROM football_data_complete ORDER BY Date;"
print('Fetching from sql')
df_raw = pd.read_sql_query(query, conn_str)
print('Fetched from mysql')
conn_str.close()

Fetching from sql
Fetched from mysql


In [8]:
df_raw = df_raw.dropna()

In [49]:
nffc_form = team_averages_per_game_and_runs('Nottm Forest', 'Derby', date(2021,2,28), date(2020,2,28), True, df_raw, 10)
nffc_form

{'ftg_for': 0.9543807664114148,
 'ftg_ag': 0.8623382406607925,
 'ftw': 0.3312985822455711,
 'ftd': 0.38486111566651515,
 'ftl': 0.2838403020879136,
 'htg_for': 0.36629792935383504,
 'htg_ag': 0.45765765953980575,
 'htw': 0.31409049459023247,
 'htd': 0.32575814506284756,
 'htl': 0.36015136034691975,
 'sot_for': 3.4652366400460233,
 'sot_ag': 3.4427211047887645,
 's_for': 12.354403730255697,
 's_ag': 10.275080512619118,
 'c_for': 5.192360283392282,
 'c_ag': 5.27264623980009,
 'win_run': 0.0,
 'winless_run': 0.0,
 'draw_run': 0.0,
 'drawless_run': 0.0,
 'losing_run': 0.0,
 'lossless_run': 0.0}

13.0

In [41]:
df_raw[['FTHG', 'FTAG']].mean(axis=1).mean()

1.313781500602812

In [42]:
df_raw[['FTHG', 'FTAG']].std(axis=1).std()

0.8264706487353691